In [ ]:
import face_recognition
import cv2
import os

known_encodings = []
known_names = []

for file in os.listdir("known_faces"):
    img = face_recognition.load_image_file(f"/Users/artemsotnikov/PycharmProjects/ML/lesson_21/known_faces/{file}")
    enc = face_recognition.face_encodings(img)[0]

    known_encodings.append(enc)
    known_names.append(os.path.splitext(file)[0])

video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()

    frame = cv2.flip(frame, 1)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    locations = face_recognition.face_locations(rgb)
    encodings = face_recognition.face_encodings(rgb, locations)

    for face_encoding, (top, right, bottom, left) in zip(encodings, locations):
        matches = face_recognition.compare_faces(known_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            index = matches.index(True)
            name = known_names[index]

        cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
        cv2.putText(frame, name, (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

from deepface import DeepFace
import cv2
import uuid

DB_PATH = "/Users/artemsotnikov/PycharmProjects/ML/lesson_21/faces_db/"

video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    frame = cv2.flip(frame, 1)  # зеркалим

    temp_path = f"/tmp/{uuid.uuid4()}.jpg"
    cv2.imwrite(temp_path, frame)  # нужно для DeepFace.find()

    try:
        result = DeepFace.find(
            img_path=temp_path,
            db_path=DB_PATH,
            model_name="ArcFace",
            detector_backend="mediapipe",
        )

        if len(result) > 0 and len(result[0]) > 0:
            identity = result[0].iloc[0]["identity"]
            name = identity.split("/")[-2]
        else:
            name = "Unknown"

    except Exception as e:
        print("Ошибка:", e)
        name = "No face detected"

    finally:
        if os.path.exists(temp_path):
            os.remove(temp_path)  # удаляем временный файл

    cv2.putText(frame, name, (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("DeepFace ArcFace webcam", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import os
import numpy as np

recognizer = cv2.face.LBPHFaceRecognizer_create()

faces = []
labels = []
names = {}
label_id = 0

# load training images
for name in os.listdir("/Users/artemsotnikov/PycharmProjects/ML/lesson_21/faces_db"):
    folder = os.path.join("/Users/artemsotnikov/PycharmProjects/ML/lesson_21/faces_db", name)
    names[label_id] = name

    for file in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, file), cv2.IMREAD_GRAYSCALE)
        faces.append(img)
        labels.append(label_id)

    label_id += 1

recognizer.train(faces, np.array(labels))

# webcam stream
video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier("/Users/artemsotnikov/PycharmProjects/ML/lesson_21/haarcascade_frontalface_default.xml")

    if face_cascade.empty():
        raise Exception("Failed to load haarcascade_frontalface_default.xml")

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        roi = gray[y:y+h, x:x+w]
        label, confidence = recognizer.predict(roi)

        name = names[label] if confidence < 80 else "Unknown"

        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, name, (x,y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    cv2.imshow("OpenCV LBPH", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
